## DEMO 7

In [ ]:
# cargo el dataset

import pandas as pd
df = pd.read_csv('T_C02-1C2025T-historico_prod.csv', sep=';')
df

In [ ]:
# hago un one hot encoding (https://www.youtube.com/watch?v=JDMViAeLxgQ)

df = pd.get_dummies(df, dtype='int')
df

In [ ]:
# elimino la columna año
df = df.drop(columns=['año'])
df

In [ ]:
# reviso la información del dataset ...
df.info()

In [ ]:
# separo X e y
y = df['ventas_und']
X = df.drop(columns=['ventas_und'])



In [ ]:
# scalo datos
from sklearn.preprocessing import StandardScaler

e = StandardScaler()
X = e.fit_transform(X)

In [ ]:
# divido el dataset en entrenamiento (80%) y test (20%)

from sklearn.model_selection import train_test_split

print('tamaño X: ', X.shape)
print('tamaño y: ', y.shape)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=9)

print('train X', X_train.shape, 'train y', y_train.shape)
print('test X', X_test.shape, 'test y', y_test.shape)

In [ ]:
import torch    # importo la biblioteca de pyTorch

device = 'cpu' # 'cpu' 'cuda' # selecciono el dispositivo que va a procesar los datos

# convierto mis arrays en tensores
X_train_t = torch.tensor(X_train).float().to(device)
X_test_t = torch.tensor(X_test).float().to(device)
y_train_t = torch.tensor(y_train.values).float().to(device)
y_test_t = torch.tensor(y_test.values).float().to(device)
y_test_t = y_test_t[:, None]
y_train_t = y_train_t[:, None]


In [ ]:
# genero mi red neuronal (otra forma donde tengo más control que en la definicion que realicé en la demo 6)



class nn1(torch.nn.Module):

    def __init__(self):
        super(nn1, self).__init__()
        self.fc1 = torch.nn.Linear(in_features=47, out_features=100, bias=True)
        self.relu1 = torch.nn.ReLU()
        self.fc2 = torch.nn.Linear(in_features=100, out_features=50, bias=True)
        self.relu2 = torch.nn.ReLU()
        self.fc3 = torch.nn.Linear(in_features=50, out_features=30, bias=True)
        self.relu3 = torch.nn.ReLU()
        self.fc4 = torch.nn.Linear(in_features=30, out_features=1, bias=True)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.fc2(x)
        x = self.relu2(x)        
        x = self.fc3(x)
        x = self.relu3(x)        
        y = self.fc4(x)
        return y

In [ ]:
# instanciamos el modelo

model = nn1()
print(model)

In [ ]:
# defino los hiperparametros de entrenamiento:

lr = 0.001
epochs = 50000
e_print = 100
loss_fn = torch.nn.MSELoss()
optimizer = torch.optim.Adam(params=model.parameters(), lr=lr)
historico = pd.DataFrame()

In [ ]:
# realizo el entrenamiento (dependiendo el procesador esta operacion tarda unos 6 minutos):

for epoch in range(1, epochs+1):
    y_pred = model(X_train_t)
    loss = loss_fn(y_pred, y_train_t)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    
    if epoch % e_print == 0:
        print(f"epoch {epoch} \t Loss: {round(loss.item(),4)}")

In [ ]:
# pruebo el resultado con un dato del dataset de TEST:


i = 2
Xp = X_test_t[i]        # pruebo con los datos de la fila i del dataset de prueba.

pred = model(Xp)        # calculo la predicción del modelo

print(f"el modelo predijo: {int(pred.item())} sin embargo el resultado debería haber sido: {int(y_test_t[i].item())}")      

In [ ]:
# pruebo con un dato nuevo

infl = 11.0 # defino una inflación del 4%

# genero el vector con los datos que quiero probar
test = [[infl, 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]]
Xp = e.transform(test)  # escalo los datos
r = torch.tensor(Xp).float() # los convierto en un tensor

pred = model(r) # calculo la predicción de modelo

print(f"el modelo predijo: {int(pred[0].item())}")